In [205]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import os

In [206]:
model_version = '1.0'
# latent factor의 수
num_features = 30
# iteration 수
iterations = 140
# learning_rate 
learning_rate = 1e-1
# lambda
lambda_ = 1
# count_weight
count_weight = 20

In [207]:
# 저장된 모델 버전 및 체크포인트 경로 설정
checkpoint_path = f'../model/{model_version}/'

# CSV 파일 불러오기
UI_temp = pd.read_csv(checkpoint_path + 'UI_temp.csv').drop('userId', axis=1)
UI_count_div = pd.read_csv(checkpoint_path + 'UI_count_div.csv').drop('userId', axis=1)
test_for_loss = pd.read_csv(checkpoint_path + 'test_for_loss.csv')
test_data_df = pd.read_csv(checkpoint_path + 'test_data_df.csv')
user_category_not_valid_df = pd.read_csv(checkpoint_path + 'user_category_not_valid.csv')
category_df = pd.read_csv(checkpoint_path + 'category.csv')

In [208]:
bins = category_df.values[0, 1:]
labels = category_df.columns[1:]

In [209]:
# labels의 dtype을 float64로 변경
labels = labels.astype('float64')

In [210]:
# CF를 위한 초기값 설정
Y = np.array(UI_temp) 
Y = Y.T
count = np.array(UI_count_div)
count = count.T
print(Y.shape)
R = Y != 0 
n_u = Y.shape[1]
n_o = Y.shape[0]

(570, 14)


In [211]:
# 기록이 존재하는 값의 평균을 구함
o_sum = Y.sum(axis=1)
o_count = R.sum(axis=1)
o_mean = o_sum / o_count
o_mean = o_mean.reshape(-1, 1)

In [212]:
Y_stand = Y - (o_mean * R)

In [213]:
# user, outfit의 수
n_o, n_u = Y.shape

# (U,O)를 초기화하고 tf.Variable로 등록하여 추적
tf.random.set_seed(1234) # for consistent results
U = tf.Variable(tf.random.normal((n_u,  num_features),dtype=tf.float64),  name='U')
O = tf.Variable(tf.random.normal((n_o, num_features),dtype=tf.float64),  name='O')
b = tf.Variable(tf.random.normal((1,          n_u),   dtype=tf.float64),  name='b')

# optimizer 초기화
optimizer = keras.optimizers.Adam(learning_rate = learning_rate)

In [214]:
def load_variables_optimizer(variables, optimizer, checkpoint_path):
    checkpoint = tf.train.Checkpoint(variables=variables, optimizer=optimizer)
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_path))

In [215]:
# 훈련된 변수 불러오기
load_variables_optimizer({"O": O, "U": U, "b": b}, optimizer,  checkpoint_path)

In [216]:
O

<tf.Variable 'O:0' shape=(570, 30) dtype=float64, numpy=
array([[ 0.00010132, -0.00298993,  0.00601141, ..., -0.00284908,
         0.00147215,  0.00494636],
       [ 0.00022328, -0.00301111,  0.00572545, ..., -0.00224591,
         0.0009839 ,  0.00580692],
       [-0.00872225,  0.00500805, -0.00547501, ...,  0.00423807,
        -0.00124386, -0.00709001],
       ...,
       [-0.00756777,  0.00470383, -0.00470229, ...,  0.00451384,
        -0.00275748, -0.00574044],
       [-0.00146283, -0.00036365,  0.00176229, ..., -0.00202911,
        -0.00229116,  0.00182291],
       [ 0.00089593, -0.00038134, -0.00109004, ...,  0.00093187,
         0.00316914, -0.00164165]])>

In [217]:
def cofi_cost_func_v(O, U, b, Y, R, lambda_):
    j = (tf.linalg.matmul(O, tf.transpose(U)) + b - Y )*R
    J = 0.5 * tf.reduce_sum(j**2) + (lambda_/2) * (tf.reduce_sum(O**2) + tf.reduce_sum(U**2))
    return J

In [218]:
def predict(O, U, b, o_mean, count, count_weight, UI_temp, labels) :
    # 예측을 수행하기 위해 모든 user-item에 대한 예측값을 계산
    p = np.matmul(O.numpy(), np.transpose(U.numpy())) + b.numpy()
    # user_category_not_valid에 해당하지 않는 경우에 대해 precision, recall, f1_score 계산
    # 평균을 위한 초기화
    for i in range(UI_temp.shape[0]):
        for category in labels:
            
            # 실제 온도
            # 평균을 적용하고 temp를 빼서 값이 작을수록 실제 온도에 가깝도록 함. 이 때 각 user-item의 사용 횟수를 가중하여 많이 사용한 item이 추천되도록 함
            pm = np.power(p + o_mean - category, 2)  -count * count_weight
            my_predictions = pm[:,i]

            # sort predictions
            ix = tf.argsort(my_predictions, direction='ASCENDING')

            df_predict = UI_temp[UI_temp.columns[ix[0:3]]].copy()
            df_predict = df_predict.round(0)
            # df_predict의 columns와 test_data_df의 '옷 조합' column을 비교하여 일치하는 경우의 개수를 계산
            predict = df_predict.columns.astype(str)
            
            # user i에 대한 예측을 파일로 저장
            os.makedirs(f'../data/predictions/male/user_{i+1}', exist_ok=True)
            # Save predictions to file in user's directory
            with open(f'../data/predictions/male/user_{i+1}/predictions_{category}.txt', 'w') as f:
                for item in predict:
                    f.write("%s\n" % item)

In [219]:
predict(O, U, b, o_mean, count, count_weight, UI_temp, labels)

In [220]:
cofi_cost_func_v(O,U,b,Y,R,lambda_)

<tf.Tensor: shape=(), dtype=float64, numpy=13051.930810247874>